# Book Recommendation System

**Team members:**  
- Deepak Maran: dm3308   
- Kewei Liu: kl2987  
- Rakshita Nagalla: rn2439   
- Xiaohui Guo: xg2225    

Part II Deep Dive  of the project for E4571 Personalization: Theory & Application.  

## Introduction
Book crossing is a website that enables free sharing of books, either with friends or with complete strangers by wild-releasing books in public places. They offer optional paid membership for new members that includes features like book recommendation. These recommendations direct to the  Better World Books website which is an online book-selling portal.  

In this context, better top-k recommendations generate revenue to Bookcrossing in two ways:  

1. Better recommendations translate to purchases at World Books and Bookcrossings gets a share.  

2. Personalized recommendations leads to user satisfaction resulting in retention of users and brand value. This contributes to revenue through paid memberships as new users would opt for the optional membership and existing users would continue to pay to retain the membership.  

Thus, our business objective is to generate k recommendations(top-k recommendation) for users logged into bookcrossings website.  

**Challenge: **  

We built a top-k recommendation engine for this task and explored neighborhood and model based collaborative filtering models for this task using collaborative filtering techniques in Part I. However, these methods are not able to recommend latest books to users as they have not been reviewed enough times. But recommending new books in the market that are relevant to the users is integral to driving sales for that book, and in turn for bookcrossing to generate revenue. In this project, we hope to design recommender system that addresses this book cold start problem.   

Thus our objective is to generate recommendations that work reasonably well even for new books in the market.   

## Dataset Source
Book crossing is a website that enables free sharing of books, either with friends or with complete strangers by wild-releasing books in public places. The data we are going to use is from [here](http://www2.informatik.uni-freiburg.de/~cziegler/BX/).  

There are three datasets.   
The BX-Users.csv contains information about 278,858 users.   
The BX-Books.csv contains information about 271,360 books.   
And the BX-Book-Rating contains information about 1,149,780 ratings, from 105,283 users and 340,556 items.

The ratings dataset contains explicit feedbacks where users explicitly rate books on a scale of 1-10. We assume that this ratings data is a proxy of the actual preference of the user.   

In addition to the ratings data we used in part I, we leverage the information about the book title, author and publisher information provided by book-crossings to deliver better recommendation and to address the cold-start problem. 

## Recommendation Task
We recommend books to the user if the book’s estimated rating $\hat{r}_{ui}$ is greater than the threshold,  and if it is among the k highest estimated ratings. Also, unlike some applications like music recommendation, users are not usually interested in repurchasing the books. We also take this into account while predicting and do not recommend book also seen in the train set.


## Evaluation metrics

### 1. Accuracy
Since our objective is to recommend $k$ items to the user, evaluation metrics like RMSE and MAE which measure accuracy of rating prediction are not very indicative of the end-user experience. To satisfy our business objective, we are are more concerned about the precision at the top. Hence we choose the following evaluation metrics:  

- **Precision**:   
    Precision measures the proportion of recommended items that are actually relevant and is directly related to our business objective. We define an item as being relevant to a user, if he gave it a rating of 7 or above. We empirically chose the threshold value to be 7 based on the ratings distribution.  

- **Recall**:   
    Recall indicates the proportion of relevant items that are in the recommended list. We consider recall also because it provides information on the expected performance of the recommender system regarding the unknown complete data (i.e., all books), which exactly is what is experienced by the user. If we assume that relevant ratings are missing at random (while allowing all other rating values to be missing not at random), then Recall can be estimated without bias from observed MNAR data.  

- **F-score**:   
    F-score summarizes the trade-off between precision and recall.  
$$F\, score = \frac{2 * Precision * Recall}{Precision + Recall}$$
  
- **NDCG**:   
    In our case, the ranking between 1st and 2nd ranked books is more important than those at the end of the list. Moreover, unlike in precision and recall, NCDG does not require relevance to be binary, allowing the user’s preference of the book to expressed in multiple levels of relevance. In order to incorporate these idea, we use the normalized discounted cumulative gain to evaluation our recommendation lists.  
    
    $$ DCG = \frac{1}{m}\sum^{m}_{u=1}\sum_{j\in I_{u}, v_{j}\leq L} \frac{2^{rel_{uj}}-1}{log_{2}(v_{j}+1)} $$
    
    $$NDCG= \frac{DCG}{IDCG}$$
    
    where IDCG is the ideal DCG, computed using the ranking of actual rating.
    Note that, we choose $rel_{u,j}$ to be the actual rating of the user $u$ gave to item $j$.

We also plot the precision-recall curves for all the three algorithms for different values of $k$.

### 2. Coverage
For a recommender system to be accurate, the recommendations should not only be accurate but also have high coverage, i.e., it should be able to recommend a good proportion of items. We look at two types of coverage:  

- **User-space Coverage**:  
    It indicates the percentage of users for whom the recommendation system is able to recommend at least k books.  
    
- **Item-space Coverage(Catalog Coverage)**:  
    It is specifically suited for recommendation lists. It is defined as the fraction of items that are recommended to at least one user.  
    
    $$CC = \frac{|\bigcup\limits_{u=1}^{m}T_u|}{total\,\,no.\,of\,items}$$  
    
    where $T_{u}$ is the recommendation list for user $u$.

### 3. Novelty
Popular books are more likely to get a high ratings and in turn are more likely to be recommended to user. However always recommending popular books can be useless to user since they may have known them and users may expect the recommendation system to recommend some books they doesn't know. Thus we look at novelty.  

Novelty is usually defined as the complement of the item's popularity in the dataset: $-log_2 p(i)$, where $p(i) = \frac{|u \in U, r_{ui} \neq \emptyset|}{|U|}$, which is the fraction of users who rated item $i$. This information is called the self-information of an item $i$, which gives more importance to very rare items.  

Then we define the novelty of a recommendation list $R_u$ for a user $u$ as the average novelty of its books:  

$$ Novelty(R_u) = \frac{\sum_{i \in R_u} -log_{2} p(i)}{|R_u|}$$

The larger the novelty, the more likely the recommendation system will recommend unknown books to users.

### 4. Serendipity
Novelty and serendipity are closely related and their definitions in the literature often overlap. Here we would like to define serendipity as the surprise of recommendations. Specifically, we define serendipity as below:  

A simple strategy for recommending books is recommend the books with highest ratings. We can define surprising books as the books that are not among highest ratings. Let a number of books with highest ratings form a primitive recommendation list $R_{PM}$. Then we define the serendipity of a recommendation list $R$ for user $u$ as the fraction of books that are not in this list:  

$$ Serendipity(R_u) = \frac{|R_{unexpected}|}{|R_u|}$$

where $R_{unexpected}$ is the subset of books in $R_u$ that are surprising to user $u$, that is, $R_{unexpected} = R_u - R_{PM}$.  

The larger the serendipity, the more likely the recommendation system will recommend surprising books to users.

## Cleaning the data
The original data contains ratings of value 0, which indicate implicit feedback. Since we will only use explicit feedbacks, we first filter out the ratings that are between 1 to 10.  

Since the website provides three dataset, containing information about users, books, and ratings, we pick out ratings whose corresponding user and book exist in the users and books dataset.  

To check whether the dataset is sparse or dense, we compute the density of the dataset. The density of a dataset means the percentage of user-book pairs that have a rating. It is calculated as :

$$Density = \frac{no.\,of\,ratings}{no.\,of\,users * no.\,of\,items}$$

We add to the dataset we worked with in Part I, by including infrequent users and less popular items. As a result, the density of the data reduces to 0.0276% from 0.8143% that we worked with for part I. This is beacuse we no longer discard new users and unpopular items. At this stage, the dataset contain 5,471 users and 12,182 items. 

## Models
We design a hybrid recommender system that generates recommendations to users. We use multiple hybridization strategies to combine predictions from a subset of the following three models:

### k-Nearest Neighbor
When implemented the k-nearest neighbor method, we found out that taking item-based similarity is seen to perform better than user-based similarity. The search for users in a large user population is the bottleneck in user-based methods. This is overcome by instead exploring relationships between items first which are smaller in number. A basic item-based kNN model with the Pearson similarity is seen to work best for our dataset.The basic kNN model was found to work best with respect to the evaluation metrics we want to optimize for. 

### 2. SVD Matrix Facotorizaton
Due to SVD’s simplicity and effectiveness when using explicit data, we choose SVD matrix factorization as one of our models. The SVD model will learn the bias, such as certain users might not be very critical and tend to rate all book highly, or some users tend to rate all book very low. We also include the global bias term in the SVD model.  

The bias bu and the factors pu are assumed to be zero when user u is known. Similarly. When an item is known, the bias bi and the factor qi are assumed to be zero.  We overcome this by adding regularization terms of the objective function. We chose the number of factors to be 10, learning rate to be 0.001, and regularization parameter of 1.

### 3. Content based models
1. Item profile creation  
    Once an item profile is created for each of the items in the dataset using the feature engineering methods described above, we estimate the rating user $u$ gave to item $i$ ($r_ui$) using a set ($I$) of $k$ items that are most similar to item $i$ among all the items user $u$ has rated.  Cosine distance between the two item profiles was used as proxy for similarity. Thus, the rating for user $u$ and item  $i$  is estimated as:  
    $$r_{ui} = b_u + \frac{\sum_{j \in I}sim(i,j)*r’_{ju}}{ \sum_{j \in I}sim(i,j)}$$
    In order to account for the bias in the users’ ratings, r’_ui is calculated by subtracting the average rating of user $b_u$ from each of his ratings.   
<br>
2. Feature engineering  
    - Publisher  
    There are 16809 unique publishers in the dataset. Since it is a nominal variable, we only consider publishers who have published more than 5 books and one-hot encode them. The reasoning behind this is that most of the book are published by a select few publishers, hence ignoring publishers who published less than 5 book reducing the dimensionality greatly without losing much information.  
    <img src="https://github.com/Kewei-Liu/E4571_Personalization_Project-Part_II/blob/master/img/Publisher_ISBN.png?raw=true">
    
    - Author  
    There are 102,024 unique authors in the dataset. Since it is a nominal variable, we only consider authors who have written more than 5 books and one-hot encode them. The reasoning is same as that of encoding the publisher.  
    <img src="https://github.com/Kewei-Liu/E4571_Personalization_Project-Part_II/blob/master/img/Author_books.png?raw=true">
    <br>
    - Title  
    Before extracting features from the title of the book, we remove numbers, punctuations, stopwords and non-ascii characters. Further we perform lemmatization instead of stemming to reduce the word to root word, since we want the root word to retain its meaning to match with the word-embeddings. In addition, all the words are converted to lower-case alphabets.  
    <br>
    Word embeddings:  
    Since we only have information about the book titles, the corpus might not be sufficient to capture the meaning of words using just the bag of words model or tf-idf features. Hence, we used the word2vec model pre-trained on Common Crawl (CC) dataset. Word2vec model is a two-layer neural net that processes text. Its input is a text corpus and its output is a set of vectors: feature vectors for words in that corpus. Given enough data, usage and contexts, Word2vec can make highly accurate guesses about a word’s meaning based on past appearances. We take the aggregate of feature vectors for each of the words in the document in order to obtain a 300-dimensional feature vector for each of the titles. Out of the 11309 words in our corpus, word vectors were obtained for 10413 words after cleaning. These feature engineering techniques result in a feature vector of length 1026 for each book.  

### Hybrid Models

Next, we explore the following hybridization strategies to combine these base models.   
Since collabortive filtering methods and content based methods both have its own prons and cons, we want to hybridize them and make recommendations. We tried three hybrid approaches: switching, weighted, and mixed. Each approach is used for two hybrid model, one hybridize kNN method with content based method, one hybridize svd-MF method with content based method.

- switching:   
    Because if the number of neighbors of a book is too small, kNN method may not produce very good predictions. Thus for kNN & content based hybrid model, if the number of neighbors of a book is less than some threshold, which we set to 2, we will use content based model to generate estimated ratings for this book.  
      
    If the number of ratings of a user is too small, we can not get much information of this user's profile so svd method may not produce very good predictions. Thus for svd-MF & content based hybrid model, if the number of ratings of a user is less than some threshold, which we set to 5, we will use content based model to generate estimated ratings for this user.  
<br>
- weighted:   
    This hybrid approach requires both the collaborative filtering method and the content based method to make predictions. Then it combine the estimated ratings into a single one by taking the weighted average. Here we set 0.5 weight to each prediction.  
<br> 
- mixed:  
    This hybrid approach combines the two predictions from collaborative filtering method and content based method by taking the top $n$ estimated ratings from each predictions, and then choose the top $k$ out of them to form the top-k recommendation. Here we set $n$ to 10.   


## Evaluation Setup
We randomly sampled 80% of the dataset to form the train set and use the remaining 20% to evaluate our metrics on.  

Since we have tuned the hyperparameters in part I, we use the same value we choose in part I for collaborative filtering models. For the content based model, we choose the similarity metric and size of neighborhood through evaluation on the test set.

## Evaluation Results

We compute the evaluation metrics for 10 models, baseline model, kNN model, kNN-content switching hybrid model, kNN-content weighted hybrid model, kNN-content mixed hybrid model, SVD-MF model, SVD-content switching hybrid model, SVD-content weighted hybrid model, and SVD-content mixed hybrid model. The baseline model predicts the baseline estimate for given user and item using the following model:

$$ \hat{r_{ui}} = \mu + b_u + b_i$$

In the following table, we report both the biased estimates of metrics on train data and unbiased estimate of metrics on held-out test data set. These metrics were calculated for a recommendation list of size 10 and the threshold for recommendation is set to 7.  

<img src="https://github.com/Kewei-Liu/E4571_Personalization_Project-Part_II/blob/master/img/Training_metric.png?raw=true">
<br>
<img src="https://github.com/Kewei-Liu/E4571_Personalization_Project-Part_II/blob/master/img/Test_metric.png?raw=true">

From the above table, we can observe that content based model gives the best novelty. And hybrid models all give a better novelty than baseline model and collaborative filtering models as we expected. The results make sense because content based recommendation system are more likely to recommendend less popular items.  

Because the content based model doesn't provide very high accuracies, it also causes the hybrid models to have lower accuracies than collaborative filtering models. By improving the content based model we may get better results of the hybrid models.  

For models use kNN, we can notice that among different hybrid methods, switching method produced very similar metrics to the content based model. This is because a lot of items have less than 2 neighbors so we used content based model to estimate ratings for corresponding user-item pairs. It also explains why kNN method has low precision and high recall. The lack of neighbors makes it less likely to give positive estimates.

For item-space coverage metrics, we can see that mixed hybrid method works pretty well. It means that by simply combine the recommendation lists of produced by two method can recommend more different items to all users.

For serendipity metrics, to some degree we can say that the weighted hybrid method is more likely to generate higher serendipity. So this method are more likely to recommend surprising books. But overall the differences between different model are not very distinct in both tables. Considering the sparicity of the dataset, we think by increasing the ratings data and primitive recommendation list we may get more informative results.

## Data Segmentation and Model Exploration

Book Popularity is measured by the number of users who have rated the book. The histogram shows a high number of unpopular books and low number of popular books.  
<img src="https://github.com/Kewei-Liu/E4571_Personalization_Project-Part_II/blob/master/img/book_popularity.png?raw=true">

User Prolificity is measured by the number of ratings a user gives. The histogram shows a large number of less prolific users and a low number of prolific users.  
<img src="https://github.com/Kewei-Liu/E4571_Personalization_Project-Part_II/blob/master/img/book_popularity.png?raw=true">

The histogram  shows the distribution of the ages of the users who have rated books. The data shows a higher number of middle-aged users compared to younger and older users.
<img src="https://github.com/Kewei-Liu/E4571_Personalization_Project-Part_II/blob/master/img/Histogram_of_Age.png?raw=true">

And the histogram of book publication year below shows that there are a lot more recently published books than older books in our dataset.

<img src="https://github.com/Kewei-Liu/E4571_Personalization_Project-Part_II/blob/master/img/Book_publication_year.png?raw=true">

The performance of the SVD switching model was explored across different segments of data.

The test dataset was split based on the following criteria separately into multiple segments and the evaluation metrics compared across them.
1. Book popularity: Less Popular and More Popular
2. Reader prolificity: Less Prolific and More Prolific
3. Reader Age: <30 years, 30-40 years, >40 years
4. Book Age (Date of Publication): <1995, 1995-2000, >2000

<img src="https://github.com/Kewei-Liu/E4571_Personalization_Project-Part_II/blob/master/img/exploration.png?raw=true">

Comparing the NDCG scores, the model gives better accuracy for older readers than for younger readers. This could be because older readers have similar tastes whereas younger readers might have varied taste.
<img src="https://github.com/Kewei-Liu/E4571_Personalization_Project-Part_II/blob/master/img/Effect_of_reader_age.png?raw=true">

Also, the model gives more accurate predictions for the ratings of older books. This could be because the older books have accumulated ratings from more number of users over the years compared to recent books that would not have.
<img src="https://github.com/Kewei-Liu/E4571_Personalization_Project-Part_II/blob/master/img/Effect_of_book_age.png?raw=true">


## Extensions
We observe that the hybrid model above slightly improves the overall quality of recommendations made to the user. However, the improvement is not consistent or significant. One of the steps towards increasing the accuracy would be to improve the content based model. The dataset provided limited information about the user, i.e, only the author, publisher and title of book. Even a short summary can provide a lot of information about the dominant themes in the book.  

In an attempt to encode this information, we retrieved book summaries of the books in our dataset from the GoodReads website. By training an LDA model with 20 topics on retrieved book summaries, we obtained the  topic distribution vector for each book. By adding these features to the existing features set, we observed that the mean squared error of the predictions slightly decreased. This can be observed from Mean Squared error values on the test set below:  
 
| | Mean Squared Error | 
|:---:|:---:|
|**With 20 topic distribution features**|38.40|
|**Without topic distribution features**|38.57|

## Conclusion
The content based model we developed performs poorly by itself but contributes to improving the novelty of recommendations when used along with the collaborative filtering techniques as a hybrid. The high novelty score indicates that the recommender system does a good job of recommending relevant items  that are not very popular to the user. However, the accuracy of prediction as measured my F-score, precision and recall has not significantly improved. This can attributed to lack of enough data for the content based model. We believe that by using additional information about the books, the content based model can be improved, leading to better performance of the hybrid model. To some extent, this is illustrated through our experiment where we used topic distribution features to improve the accuracy of the the content based model by adding side information from a third party source.   

Moreover,  our model performs reasonably well on recently published and less popular books as well, suggesting the advantage of the hybrid model. Another business insight that we gain is that predictions are better on older users than younger counterparts. This could be due to less variance in the book preferences of an older user making it easier to model their recommendations using a content based model. This suggests that choosing hybrids wisely for users of different segments, can add value to the  business.  

## Reference

1. George Lekakos, Petros Caravela. A Hybrid Approach for Movie Recommendation. Multimedia Tools & Applications, 2008, 36 (1-2) :55-70.
2. M Kaminskas, D Bridge. Diversity, Serendipity, Novelty, and Coverage: A Survey and Empirical Analysis of Beyond-Accuracy Objectives in Recommender Systems. ACM Trans. Interact. Intell. Syst.2016, 7(1), Article 2:1-42.
3. Sridharan Suboojitha. Introducing Serendipity in Recommender Systems Through Collaborative Methods. Open Access Master's Theses. 2014, Paper 45.